# Title

## Abstract

## Table of Contents

## Related Work

## Introduction of Graph Databases & Graph Analytics Systems
_Sources:_
* _https://engineering.fb.com/2015/06/02/core-data/recommending-items-to-more-than-a-billion-people/_
* _https://arxiv.org/abs/2103.03227_
* _https://arxiv.org/pdf/2102.13613.pdf_
* _https://www.scitepress.org/papers/2018/69102/69102.pdf_
* _https://medium.com/@gauravsarma1992/neo4j-storage-internals-be8d150028db_
* _https://www.slideshare.net/thobe/an-overview-of-neo4j-internals_
* _https://arxiv.org/pdf/2102.13613.pdf_
* _https://arxiv.org/pdf/2012.06171.pdf_

Many systems of academic or economic interest give rise to datasets that can most naturally be described by graphs. Examples include the interaction between agents in a social network, content recommendation , financial crime and fraud detection as well as transaction processing. In order to efficiently store graph data, specific database designs are required that augment existing NoSQL design paradigms to better fit graph datasets.

For the purposes of this paper, a graph database system shall be defined as follows:
__Graph database systems__ are a subgroup of schemaless NoSQL database systems. Contrary to relational database systems, where the single most basic unit of data information is a record in a predefined relation, graph database systems distinguish two basic units of information: 
1. Nodes, representing graph vertices (e.g. teachers, students, etc.)
2. Relationships, representing graph edges that represent the relationship between nodes (e.g. teaches, hears, tutors, etc.)

Both nodes and relationships allow for the classical notion of attributes in relational databases, though without a fixed schema. The aforementioned types of nodes and relationships are called labels and each node or relationships can contain one label value (e.g. a node is labeled as a teacher or student). By inheritance of the characteristics of schemaless databases as well as their augmentation through the additional structures outlined above, the following core characteristics of graph databases shall be noted.

* __Flexible schema__ - While relational databases require new tables or alterations in the existing ones to add new types of data, in a graph database we can add new type of vertices and edges without alterations in the previously stored data;  
* __Query language__ – Relational databases use SQL query language to manipulate the database, but graph databases needed a more powerful query language. There are many different query languages to manipulate specific graph databases like Cypher on Neo4J or AQL on ArangoDB, and we will study to see which one suits best the needs of querying data;  
* __Sharding__ - One of the key elements to being able to scale NOSQL databases is sharding, where a large dataset can be broken up and distributed across a number of (typically replicated) shards;  
* __Backups__ – Graph databases should provide functions for planning, performing and restoring a database backup. A full backup contains all data files and information required to restore a repository to the state it was in at the time of the backup;  
* __Multi-model__ – A multi-model graph database provides a database with unstructured data, and we can visualize relationships with data like in the form of graphs, key-value pairs, documents or tables; 
* __Multi-architecture__ - When planning a graph database solution, there are several structural decisions to make. These decisions may vary slightly, depending on which database product you choose. 
* __Scalability__ - As hardware continues to innovate at a rapid pace (mass storage devices, CPU and networks), this in turn leads to increased capacity for existing database software, enabling growth in the number of transactions per second. There are two approaches to scaling a database: Vertical (Scale Up) and Horizontal (Scale Out). Vertical scaling involves adding more physical or virtual resources to the underlying server hosting the database – more CPU, more memory or more storage. Horizontal scaling involves adding more instances/nodes of the database to deal with increased workload;
* __Cloud ready__ – A graph database implemented on cloud is a great feature because it solves scalability problems and provides real-time management.

In the following paragraph some of the technical details of graph databases (in this case Neo4j serves as a representative implementation) will be outlined. As will be detailed in the following sections, a graph generally consists of a set of nodes and edges, both of which can contain a set of attributes. This set of attributes can differ for each node/edge.

Each node is stored as an individual node record. Node attributes are stored as a singly-linked lists consisting of key-value pairs, with the first (ordering is arbitrary and subject to insertion order unless explicitly specified otherwise) list item being pointed to by the node record itself. Node records also contain a pointer to the first (again, ordering is arbitrary unless specified otherwise) edge the node is referenced by.
Similarly, edges, in the language of Neo4j referred to as relationships, are stored as individual records. Just like nodes, the attributes of a relationship are stored as singly-linked lists. A relationship record contains not only pointers to its start- and end-node, but also to the next/previous relationship of its end-/start-node respectively (again, ordering is arbitrary). This data structure allows for “index free adjacency”, meaning there is no need for looking up indices to find relationships between nodes. This marks one major advantage over representing graphs in relational database systems, as such joins tend to become computationally expensive on large datasets.

The data types described above are persisted in separate stores on disk, namely Node Stores, Relationship Stores and Property Stores. In the case of Neo4j, large attribute values (e.g. BLOBs) are stored in yet again separate files and merely a pointer to the respective file is stored directly as the value in the singly-linked list.

Other peripheral and implementation specific software modules, such as caching, high availability functionality etc. shall not be detailed in the course of this paper.

__Graph Analytics Systems__ operate on top of a (graph-) database and allow for efficient analysis of the underlying data. Most of these systems provide interfaces to common (graph-) databases for frictionless data integration and preprocessing  as well as frequently used graph algorithms.

Major implementations include Apache Giraph, GraphLab, Blogel and Spark GraphX. The latter will be the focus of this paper, as well as it’s successor, GraphFrames.


## Theory & Analysis of Graph Datasets

### Introduction to Graph Theory

Depending on the inherent structure of an individual dataset different types of graphs are required to encode its relevant attributes. Some of the most prevalent graph structures include: 

__Undirected graphs__
An undirected graph is defined as an ordered pair $ G = (V, E) $, with:
* $ V $, a set of vertices
* $ E $, a set of edges, where each edges is an unordered pair of vertices

Undirected graphs are commonly used to represent symmetric associations between entities, e.g. the connectedness of cities given a specific road network. In this example each node represents a city and each edge represents a road between two cities.
In the example above it is assumed that each road can be traversed in both directions. In order to represent the more general case of including one-way roads in the road network, one needs to augment undirected graphs with directionally encoding.

__Directed graph__
A directed graph is defined as an ordered pair $ G = (V, E) $, with:
* $ V $, a set of vertices
* $ E $, a set of edges, where each edges is an ordered pair of vertices

Coming back to the previous example, a directed graph allows for the definition of unidirectional roads between cities, e.g. due to road width restrictions.
Given the ability to represent all road networks as a directed graph, one might want to perform analyses on upcoming trips. For example, one might be interested in finding the shortest path connecting two cities. To do so, information about distances between cities needs to be incorporated in the graph data structure.

__Weighted directed graph__
A directed graph is defined as an ordered pair $ G = (V, E, \phi) $, with:
* $ V $, a set of vertices
* $ E $, a set of edges, where each edges is an ordered pair of vertices
* $ \phi $ a function that associates a real value to each edge

Completing the aforementioned example, Phi associates a distance measure to each edge, representing the length of each connecting road.

Finally, an algorithm for finding the shortest paths is required to make use of the data structures defined above. The following table presents a selection of these algorithms.


### PageRank Algorithm
_Sources:_
* _https://blogs.cornell.edu/info2040/2014/11/03/more-than-just-a-web-search-algorithm-googles-pagerank-in-non-internet-contexts/_
* _https://towardsdatascience.com/pagerank-algorithm-fully-explained-dc794184b4af_

In the following section the PageRank algorithm will be introduced. This algorithm is most commonly applied in order to identify “important” nodes in graph networks based on graph topology. Practical use-cases of include:
* Ranking websites by the amount of references to them weighted by reference quality
* Ranking accounts in a transaction network by their transaction activity weighted by transacted value
* Predicting traffic flow in road networks

To facilitate the weighting of nodes, the PageRank algorithm calculates the stationary state probability distribution of a Markov-Process defined by the given graph network. As a simple example, the following graph shall be regarded. This example assumes the reader to be familiar with the general concepts of Markov-Processes.

<center><img src="01_Ressources/2021-04-04_SampleGraph.PNG" height="400" width="400"/></center>

The transition matrix of the Markov-Process is set to $ 1/n $ for each outgoing edge of a node, where n is defined as the total count of all outgoing edges of the respective node . Self references are ignored and in a context of weighted edges the transition probabilities are weighted according to the total outgoing edge weights instead. Therefore, all column sums equal one and describe the probability of moving from a specific column-state to the respective row-state. The graph displayed above is represented by the transition matrix $ P $.

<center><img src="01_Ressources/2021-04-04_SampleTransitionMatrix.PNG"  height="300" width="300"/></center>

For the state probability distribution $ \pi $ to be stationary, the following must hold:

<center>$ \pi = P\pi $</center>

This implies that π must be an eigenvector of $ P $ with the corresponding eigenvalue of $ 1 $. To compute $ \pi $, one can make of the power method algorithm, which in turn makes use of the _Frobenius-Perron_ theorem. This theorem states that given a non-negative square matrix, there exists only one positive eigenvalue whose eigenvector can be chosen to consist of only positive entries, qualifying this eigenvector as the probability distribution of interest. This eigenvalue also is dominant, meaning it is the eigenvalue of $ P $ with the highest absolute value. Both implications outlined above allow one to use the Power method outlined below to find the eigenvector of interest, where vector b is initialized randomly and $ A=P $.

<center><img src="01_Ressources/2021-04-04_SampleAlgorithm.PNG"  height="300" width="300"/></center>

## Introduction of selected Graph Database & Graph Processing Systems

### Apache Spark
_Sources:_
* _Salloum et. al., 2016, "Big data analytics on Apache Spark"_

Apache Spark (short: Spark) is an In-Memory distributed computing framework that is implemented in Scala and serves similar purposes as Apache Hadoop. Though, unlike Apache Hadoop, Spark is performing processing in RAM, making it orders of magnitude faster than its On-Disk counterpart. Spark (in the narrow sense) consists of Spark Core and Upper-Level Libraries. Spark Core handles communication to a Cluster Manager in the context of data ingestion from supported databases. It also provides the Resilient Distributed Dataset (RDD) API, which is consumed by Upper-Level Libraries. Upper-Level Libraries provide functionalities built upon the RDD abstraction that allow users to e.g. instruct Spark to load a dataset and perform analysis on the dataset.

<center><img src="01_Ressources/2021-04-04_SparkStack.PNG" height="700" width="700"/></center>

An RDD is an immutable data structure that Spark uses to store (intermediate) results of processing pipelines. As the name would suggest, an RDD is partitioned across Spark Worker nodes and implements automatic fault recovery. Users can explicitly decide whether to store an RDD in memory or on disk of the respective worker nodes.
Another data structure in Spark is the DataFrame, which is built on top of the RDD data structure. As the name suggests, a DataFrame is defined in a similar fashion as e.g. Pandas DataFrames, meaning it consists of tabular data in labeled columns with predefined types. This additional information contained in the data allows Spark to further optimize data partitionining and therefore claims to boost performance.

### Apache Spark GraphX
Sources:
* https://amplab.cs.berkeley.edu/wp-content/uploads/2014/09/graphx.pdf
* [https://spark.apache.org/docs/2.1.0/graphx-programming-guide.html#pregel-api](https://spark.apache.org/docs/2.1.0/graphx-programming-guide.html#pregel-api)

In general GraphX is a component in Apache Spark technology eco-system for graphs and graph-parallel computation. GraphX reuses Spark RDD concept and provides  API for fast and robust development of graphs logic. Therefore, this integrated component simplifies graph analytics tasks and provides the ability to make operations on a directed multi-graphs.
GraphX API implements a variant of the Pregel paradigm as well as a range of common graph operations.  GraphX API enables the composition of graphs with unstructured and tabular data and permits the same physical data to be viewed both as a graph and as collections without data movement or duplication.
Before we describe GraphX more detailed, it make sense to say a couple of words about Pregel. Pregel is a data flow paradigm and system for large-scale graph processing created at Google to solve problems that are hard or expensive to solve using only the MapReduce (e.g. Apache Hadoop) framework. Pregel is essentially a message-passing interface constrained to the edges of a graph. The idea is to ”think like a vertex” — algorithms within the Pregel framework are algorithms in which the computation of state for a given node depends only on the states of its neighbors.

#### GraphX Programming Abstraction
Source: GraphX: Graph Processing in a Distributed Dataflow Framework paper Berkley https://amplab.cs.berkeley.edu/wp-content/uploads/2014/09/graphx.pdf
*Property graph as a collection* - The vertex collection contains the vertex properties uniquely keyed by the vertex identifier. The edge collection contains the edge properties keyed by the source and destination vertex identifiers. By reducing the property graph to a pair of collections it is possible to compose graphs with other collections in a distributed dataflow framework like Apache Spark.
*Graph Computation in distributed dataflow framework* - The normalized representation of a property graph as a pair of vertex and edge property collections allows embedding graphs in Apache Spark framework. A key stage in graph computation is constructing and maintaining the so-called triplets view, which consists of a three-way join between the source and destination vertex properties and the edge properties. Graph-parallel computation in such a framework as Apache Spark is expressed as a sequence of join stages and group-by stages punctuated by map operations. Consequently, these stages are the focus of performance optimization in graph processing systems. 
*GraphX Operators* - The GraphX programming abstraction extends the Spark dataflow operators by introducing a small set of specialized graph operators e.g. Graph, vertices, edges, triplets, mrTriplets, in case of Pregel API - messages. This operators will be described more detailed in the following sections.

#### GraphX built on Apache Spark
GraphX achieves performance parity with specialized graph processing systems by recasting the graph-specific optimizations on top of a small set of standard dataflow operators in Apache Spark. 
*Distributed Graph Representation.* As already mentioned GraphX represents graphs internally as a pair of vertex and edge collections built on the Spark RDD abstarction. These collections introduce indexing and graph-specific  partitioning as a layer on top of RDDs. 
The vertex collection is hash-partitioned by the vertex IDs. Whereas, the edges are divided into three edge partitions by applying a partition function (e.g., 2D Partitioning). Co-partitioned with the vertices, GraphX maintains a routing table encoding the edge partitions for each vertex.
GraphX inherits the immutability of Spark RDDs and therefore all graph operators logically create new collections rather than destructively modifying existing ones. As a result, derived vertex and edge collections can often share indices to reduce memory overhead and accelerate local graph operations.
*Implementation of  the Triplets View.* Because the vertex and edge property collections are partitioned independently, the join requires data movement. GraphX performs the three-way join by shipping the vertex properties across the network to the edges, thus setting the edge partitions as the join sites.
Furthermore, GraphX introduces a multicast join in which each vertex property is sent only to the edge partitions that contain adjacent edges. The routing table which is associated with the edge collection and constructed lazily upon first instantiation of the triplets view is used for multicast join.
As iterative graph algorithms often modify only a subset of the vertex properties in each iteration. Therefore, incremental view maintenance is applied to the triplets view to avoid unnecessary movement of unchanged data. After each graph operation, we track which vertex properties have changed since the triplets view was last constructed. When the triplets view is next accessed, only the changed vertices are re-routed to their edge-partition join sites and the local mirrored values of the unchanged vertices are reused. This functionality is managed automatically by the graph operators.
GraphX incorporates two additional query optimizations for the mrTriplets operator: filtered index scanning and automatic join elimination as well as additional optimizations for memory-based shuffle, batching and columnar structure and variable integer encoding, which we will not describe in details in this study project.

### GraphX in Apache Spark 3.1.1
Source:https://spark.apache.org/docs/latest/graphx-programming-guide.html#:~:text=GraphX%20is%20a%20new%20component,to%20each%20vertex%20and%20edge
In the following based on an example we will demonstrate how to build a property graph, how to  use some common Graph Operators and Property Operators, how to compute degree information as well as an simple application of PageRank algorithm using GraphX in Apache Spark 3.1.1.
For this example two data sets were taken from [Kaggle - online community of data scientists and machine learning practitioners](https://www.kaggle.com/). The datasets contain information about several bike stations in USA and trips which were done between this station in 2013-2014. 
Code is written is Scala, therefore should be executed in spylon-kernel. If the current notebook kernel is not spylon-kernel switch it in the upper right coner of the grafical interface.


In [1]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame

//val spark = SparkSession.builder.appName("Simple Application").getOrCreate()  
//spark context is already availiable in the running environment
//read in station dataset from csv file and register it as a table  
val station = spark.read.format("csv")
         .option("header", "true") //first line in file has header
         .option("mode", "DROPMALFORMED")
         .load("./graphX_data/station.csv")
station.registerTempTable("station")
val bikeStations = spark.sqlContext.sql("SELECT * FROM station") 
bikeStations.printSchema()

//read in trip dataset from csv file and register it as a table  
val trip = spark.read.format("csv")
         .option("header", "true") //first line in file has header
         .option("mode", "DROPMALFORMED")
         .load("./graphX_data/trip.csv")
trip.registerTempTable("trip")
val tripData = spark.sqlContext.sql("SELECT * FROM trip")
tripData.printSchema()

//make sure each station occur only ones as stations will be the nodes
val justStations = bikeStations
  .selectExpr("float(id) as station_id", "name")
  .distinct()

//station ids  
val stations = tripData
  .select("start_station_id").withColumnRenamed("start_station_id", "station_id")
  .union(tripData.select("end_station_id").withColumnRenamed("end_station_id", "station_id"))
  .distinct()
  .select(col("station_id").cast("long").alias("value"))

stations.take(1) // this is just a station_id

//create set of vertices with properties
val stationVertices: RDD[(VertexId, String)] = stations
  .join(justStations, stations("value") === justStations("station_id"))
  .select(col("station_id").cast("long"), col("name"))
  .rdd
  .map(row => (row.getLong(0), row.getString(1))) 

stationVertices.take(1)

//create trip edges
val stationEdges:RDD[Edge[Long]] = tripData
  .select(col("start_station_id").cast("long"), col("end_station_id").cast("long"))
  .rdd
  .map(row => Edge(row.getLong(0), row.getLong(1), 1))
//add dummy value of 1
stationEdges.take(1)

//build a graph
val defaultStation = ("Missing Station") 
val stationGraph = Graph(stationVertices, stationEdges, defaultStation)
stationGraph.cache()

// sanity check      
println("Total Number of Stations: " + stationGraph.numVertices)
println("Total Number of Trips: " + stationGraph.numEdges)
println("Total Number of Trips in Original Data: " + tripData.count)    

Intitializing Scala interpreter ...

Spark Web UI available at http://spark-demo-68b9d46d9-n68t4:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1617559000593)
SparkSession available as 'spark'


root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- dock_count: string (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

Total Number of Stations: 70
Total Number of Trips: 669959
Total Number of Trips in Original Data: 669959


import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame
station: org.apache.spark.sql.DataFrame = [id: string, name: string ... 5 more fields]
bikeStations: org.apache.spark.sql.DataFrame = [id: string, name: string ... 5 more fields]
trip: org.apache.spark.sql.DataFrame = [id: string, duration: string ... 9 more fields]
tripData: org.apache.spark.sql.DataFrame = [id: string, duration: string ... 9 more fields]
justStations: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [station_id: float, name: string]
stations: org.apache.spark.sql.DataFrame = [value: bigint]
stationVertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String)] = ...


In [2]:
// Most common distinations from location to location
// Demonstrate usege of the triplets 
stationGraph
    .groupEdges((edge1, edge2) => edge1 + edge2)
    .triplets.sortBy(_.attr, ascending=false)
    .map(triplet => "There were " + triplet.attr.toString + " trips fom " + triplet.srcAttr + " to " + triplet.dstAttr + "." )
    .take(10)
    .foreach(println)

There were 6216 trips fom San Francisco Caltrain 2 (330 Townsend) to Townsend at 7th.
There were 6164 trips fom Harry Bridges Plaza (Ferry Building) to Embarcadero at Sansome.
There were 5041 trips fom Townsend at 7th to San Francisco Caltrain (Townsend at 4th).
There were 4839 trips fom 2nd at Townsend to Harry Bridges Plaza (Ferry Building).
There were 4357 trips fom Harry Bridges Plaza (Ferry Building) to 2nd at Townsend.
There were 4269 trips fom Embarcadero at Sansome to Steuart at Market.
There were 3967 trips fom Embarcadero at Folsom to San Francisco Caltrain (Townsend at 4th).
There were 3903 trips fom Steuart at Market to 2nd at Townsend.
There were 3627 trips fom 2nd at South Park to Market at Sansome.
There were 3622 trips fom San Francisco Caltrain (Townsend at 4th) to Harry Bridges Plaza (Ferry Building).


In [3]:
// as directed graph is used in- and out-degrees of the vertices can be calculated
// it canbe interpreted as number of trips that arrive on a particular station or departure from it
stationGraph
  .inDegrees // computes in-degree
  .join(stationVertices)
  .sortBy(_._2._1, ascending=false)
  .take(10)
  .foreach(x => println(x._2._2 + " has " + x._2._1 + " in degrees."))

stationGraph
  .outDegrees // out-degree
  .join(stationVertices)
  .sortBy(_._2._1, ascending=false)
  .take(10)
  .foreach(x => println(x._2._2 + " has " + x._2._1 + " out degrees."))

San Francisco Caltrain (Townsend at 4th) has 63179 in degrees.
San Francisco Caltrain 2 (330 Townsend) has 35117 in degrees.
Harry Bridges Plaza (Ferry Building) has 33193 in degrees.
Embarcadero at Sansome has 30796 in degrees.
2nd at Townsend has 28529 in degrees.
Market at Sansome has 28033 in degrees.
Townsend at 7th has 26637 in degrees.
Steuart at Market has 25025 in degrees.
Temporary Transbay Terminal (Howard at Beale) has 23080 in degrees.
Market at 4th has 19915 in degrees.
San Francisco Caltrain (Townsend at 4th) has 49092 out degrees.
San Francisco Caltrain 2 (330 Townsend) has 33742 out degrees.
Harry Bridges Plaza (Ferry Building) has 32934 out degrees.
Embarcadero at Sansome has 27713 out degrees.
Temporary Transbay Terminal (Howard at Beale) has 26089 out degrees.
2nd at Townsend has 25837 out degrees.
Steuart at Market has 24838 out degrees.
Market at Sansome has 24172 out degrees.
Townsend at 7th has 23724 out degrees.
Market at 10th has 20272 out degrees.


In [4]:
// PageRank
// returns 10 most significant stations
val ranks = stationGraph.pageRank(0.0001).vertices
ranks.join(stationVertices).sortBy(_._2._1, ascending=false).take(10).foreach(x => println(x._2._2))

San Jose Diridon Caltrain Station
San Francisco Caltrain (Townsend at 4th)
Mountain View Caltrain Station
Redwood City Caltrain Station
San Francisco Caltrain 2 (330 Townsend)
Harry Bridges Plaza (Ferry Building)
Embarcadero at Sansome
2nd at Townsend
Market at Sansome
Townsend at 7th


ranks: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[954] at RDD at VertexRDD.scala:57


### Other Graph Database Systems compared to Spark GraphX <a name="otherSystems"></a>
In the following section GraphX is compared to othe graph database systems. 
Neo4J is a graph database wich was covered in the lecture. The main differencies between Neo4J and GraphX are addressed in the following. As GraphX is not a graph database but graph processing system, it should be rather used as a complement to Neo4J and not as an opposite to it. The possible ways for integration of the two systems are described.
Furthermoe, another graph processing system - Giraph is introduced. Giraph and GrapX are comparable systems and three research studies on performance comparison are represented in the following section.

#### Neo4j
Sources: 
* Graph Databases Comparison: AllegroGraph, ArangoDB, InfiniteGraph, Neo4J, and OrientDB [https://www.scitepress.org/papers/2018/69102/69102.pdf](https://www.scitepress.org/papers/2018/69102/69102.pdf)
* Research on architecture and query performance based on distributed graph database Neo4j [https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6703387](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6703387)
* Graph Databases: Neo4j Analysis  https://www.scitepress.org/Papers/2017/63560/63560.pdf 
* [https://livebook.manning.com/book/spark-graphx-in-action/chapter-1/66](https://livebook.manning.com/book/spark-graphx-in-action/chapter-1/66)

Neo4J is an open-source graph database implemented in Java. The developers describe Neo4J as a fully transactional database and a persistent Java engine where they can store structures in the form of graphs instead of tables. Neo4J uses natives graph storage which provides the freedom to manage and store data in a highly disciplined manner. 
The Neo4j can run in the embedded mode, as well as in server mode. Embedded mode must be understood as an instance that stores all the information on the disk, and not as an in-memory database, that works exclusively with the internal memory of the machine. The embedded Neo4j is ideal for hardware devices, desktop applications, and for embedded applications in servers. Running Neo4j in server mode guarantees a more common way to implement a database, which is what is most used today. On each server there is an embedded instance of Neo4j. Neo4J has many competitive advantages, which makes this software one of the most used ones in this area.
The major features of Neo4J are the following:  flexible schema, Cypher query language, HHTP API to manage the database, inde support using Apache Lucence, drivers support for Java, Spring, Scala, JavaScript, online backups, exporting of query data to JSON and XML formats, most active graph community in the world, easy to learn and to use, whiteboard-friendly data modelling to simplify the development cycle.
Neo4J has community and commercial versions.  In the community versio Neo4J doesn’t support sharding and has some limitations on the number of nodes, relationships and properties.

##### Cypher Query Language
Source: [https://neo4j.com/developer/cypher/](https://neo4j.com/developer/cypher/)
Cypher is Neo4j’s graph query language that allows users to store and retrieve data from the graph database. Cypher aims to make querying graph data easy to learn, understand, and use for everyone, but also incorporate the power and functionality of other standard data access languages. 
Cypher’s syntax provides a visual and logical way to match patterns of nodes and relationships in the graph. It is a declarative, SQL-inspired language for describing visual patterns in graphs using ASCII-Art syntax. Through Cypher, users can construct expressive and efficient queries to handle needed create, read, update, and delete functionality.
Cypher is  open source. The [openCypher](http://opencypher.org/) project provides an open language specification, technical compatibility kit, and reference implementation of the parser, planner, and runtime for Cypher.

##### Neo4J not vs. but rather with Spark GraphX
Sources: 
* [https://neo4j.com/developer/apache-spark/](https://neo4j.com/developer/apache-spark/)

As already mentioned, Apache Spark GraphX is not a graph database. Instead, it’s a graph processing system, which is useful, for example, for fielding web service queries or performing one-off, long-running standalone computations. Because GraphX isn’t a database, it doesn’t handle updates and deletes like Neo4j, which is graph databases. Graph processing systems are optimized for running algorithms on the entire graph in a massively parallel manner, as opposed to working with small pieces of graphs like graph databases.
Therefore, Neo4J is not an opposite technology to GraphX and can be used rather as a complement. 
For example, when Neo4J needs to process the very large data-sets and real time processing to produce the graphical results/representation it needs to scale horizontally. In this case combination of Neo4J with Apache Spark will give significant performance benefits in such a way Spark will serve as an external graph compute solution.
Neo4J can be integrated with Spark in a variety of ways. It can be used to pre-process (aggregate, filter, convert) raw data to be imported into Neo4j. Spark GraphX can also serve as graph processing solution. So, data of selected subgraphs can be exported from Neo4J to Spark, where analytic spects are computed. The computation results are written back to Neo4J and can be used for further operations and Cypher queries.

#####  Neo4j Connector for Apache Spark
Sources:
* [https://neo4j.com/developer/spark/overview/](https://neo4j.com/developer/spark/overview/)
* [https://neo4j.com/blog/neo4j-3-0-apache-spark-connector/](https://neo4j.com/blog/neo4j-3-0-apache-spark-connector/)

The Neo4j Connector for Apache Spark is intended to make integrating graphs together easy. There are effectively two ways of using the connector:
-   As a data source: read any set of nodes or relationships as a DataFrame in Spark;
-   As a sink: write any DataFrame to Neo4j as a collection of nodes or relationships, or alternatively use a Cypher statement to process records in a DataFrame into the graph pattern of your choice. 
The connector currently supports: RDD, DataFrames, GraphX, GraphFrames.

##### Neo4J - Mazerunner
Source: https://neo4j.com/developer/apache-spark/#mazerunner

Another integration solution is Neo4J Mazerunner, which allows exporting dedicated datasets, e.g. node or relationship-lists to Apache Spark. It supports PageRank, Closeness Centrality, Betweenness Centrality, Triangle Counting, Connected Components, Strongly Connected Components algorithms. 
After running graph processing algorithms in GraphX the results are written back concurrently and transactionally to Neo4j. One focus of this approach is on data safety, that’s why it uses a persistent queue (RabbitMQ) to communicate data between Neo4j and Apache Spark.
The infrastructure is set up using Docker containers, there are dedicated containers for Spark, RabbitMQ, HDFS and Neo4j with the Mazerunner Extension.


#### Spark Graph Module
Source: [https://databricks.com/session\_eu19/graph-features-in-spark-3-0-integrating-graph-querying-and-algorithms-in-spark-graph-continues](https://databricks.com/session_eu19/graph-features-in-spark-3-0-integrating-graph-querying-and-algorithms-in-spark-graph-continues)
Furthermore Apache Spark promises an integration of the Neo4J's query language - Cypher into Sparkecosystem. According to the articale from Databricks Spark 3 introduces a new module: Spark Graph. Spark Graph adds the popular query language Cypher, its accompanying Property Graph Model and Graph Algorithms to the data science toolbox. 
Spark Graph interacts with Spark SQL and openCypher Morpheus, a Spark Graph extension allows to manage multiple graphs and provides built-in Property Graph Data Sources for the Neo4j graph database as well as Cypher language extensions.

#### Apache Giraph
Sources:
* [https://giraph.apache.org/intro.html](https://giraph.apache.org/intro.html)
* [https://www.facebook.com/notes/facebook-engineering/scaling-apache-giraph-to-a-trillion-edges/10151617006153920/](https://www.facebook.com/notes/facebook-engineering/scaling-apache-giraph-to-a-trillion-edges/10151617006153920/)
* [https://arxiv.org/pdf/1806.08082.pdf](https://arxiv.org/pdf/1806.08082.pdf)

*Apache Giraph* is an iterative graph processing system built for high scalability. It is originated as the open-source counterpart to Pregel. Giraph is an iterative graph processing framework, built on top of Apache Hadoop. 
Giraph adds several features beyond the basic Pregel model, including master computation, sharded aggregators, edge-oriented input, out-of-core computation, and more. Giraph is a computing platform, and therefore it needs to interface with external storage in order to read the input and write back the output of a batch computation. 

#### Comparison on Performance, Scaling and Fault tolerance
Sources:
* https://amplab.cs.berkeley.edu/wp-content/uploads/2014/09/graphx.pdf
* [https://engineering.fb.com/2016/10/19/core-data/a-comparison-of-state-of-the-art-graph-processing-systems/](https://engineering.fb.com/2016/10/19/core-data/a-comparison-of-state-of-the-art-graph-processing-systems/)
* [https://arxiv.org/pdf/1806.08082.pdf](https://arxiv.org/pdf/1806.08082.pdf)

The underlying Apache Spark technology allows to horizontally scale GraphX which brings obvious performance benefit. A performance study was conucted at Berkley University. The results of the study showed that increasing number of machines from 8 to 32 caused 3x time performance speedup. In regard to fault tolerance Apache Spark brings a benefit as well as it provides lineage-based fault tolerance with negligible overhead as well as optional dataset replication. 
Another quantitative and qualitative benchmarking study was conducted by Facebook in 2016 GraphX was compared to Giraph. The Facebook researchers were especially interested in measuring the relative performance and ability of the two systems to handle large graphs. While performance was the main focus, they also studied usability and how easy it is to develop applications in the two systems. 
The experimental setup consists of executing PageRank, Connected Components, and Triangle Counting implementations on various datasets exhibiting different degrees of skewness in their graphs, namely Facebook and Twitter datasets.
Based on these experiments the authors conclude that Apache Giraph is the overall more efficient graph processing platform, being able to process up to 50x larger graphs than GraphX, while GraphX is better suited to rapid development of small applications. 
The main advantages of the GraphX platform are made out to be it’s efficiency in terms of development speed and integration efforts. GraphX can read and preprocess data from Hive via an SQL-like interface, whereas Apache Giraph requires extra integration efforts. In addition, GraphX’s shell interface allows for rapid testing of simple applications.
Experimental analysis of distributed graph systems was contucted in University of Waterloo in 2018. In this study Öszu et al. perform an extensive comparison of graph processing system architectures, focusing on MapReduce (e.g. GraphX, HaLoop) and Bulk Synchronous Processing (BSP; e.g. Apache Giraph, Blogel). The authors determine performance statistics on workloads including PageRank, calculation of weakly connected components and single source shortest paths on datasets including Twitter and World road Network. The authors report an overall superiority of those systems utilizing BSP architecture over MapReduce. Of those BSP systems Blogel-V performed best in most setups. It is stated that GraphX is not efficient for graph workloads or datasets that require large number of iterations. Futhermore, the authors point out that GraphX suffers from load balancing issues when scaling to large datasets. Graphframes is suggested as an alternative, offering more efficient implementations of algorithms for specific tasks.

## Conclusion